<a href="https://colab.research.google.com/github/hfarazul/Claude-Tools-Exercises/blob/main/Claude_wiki_research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install anthropic
!pip install wikipedia

import anthropic
import os
import wikipedia

client = anthropic.Anthropic( # defaults to os.environ.get("ANTHROPIC_API_KEY")
api_key="your-api-key",
)

os.makedirs('output', exist_ok=True)
open('output/research_reading.md', 'w').close()

In [31]:
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")


generate_wikipedia_reading_list_tool = {
    "name": "generate_wikipedia_reading_list",
    "description": "Generates a list of Wikipedia articles and their URLs based on a provided list of potential article titles for a given research topic.",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {"type": "string", "description": "The research topic."},
            "article_titles": {"type": "array", "items": {"type": "string"}, "description": "A list of potential Wikipedia article titles."}
        },
        "required": ["topic", "article_titles"]
    }
}




def get_research_help(topic, num_articles=3):
    prompt = f"""You are a research assistant helping me gather information on the topic of "{topic}". I need you to generate a list of {num_articles} potential Wikipedia article titles related to this topic that I could read.

However, I want you to use a specific tool to generate the list of article titles. You must respond with a JSON object in the following format:

{{
    "tool": "generate_wikipedia_reading_list",
    "topic": "<research_topic>",
    "article_titles": ["<article_title_1>", "<article_title_2>", ...]
}}

Replace <research_topic> with the research topic "{topic}" and provide a list of {num_articles} potential Wikipedia article titles in the "article_titles" field.

You should not provide the article titles directly in your response. Instead, you must use the JSON format above to invoke the "generate_wikipedia_reading_list" tool and generate the list of potential article titles.

Please respond with the JSON object now to invoke the tool and generate the list of potential article titles.
"""
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system="You have access to tools, but only use them when it's helpful. If a tool is not required, respond as normal",
        messages=[{"role": "user", "content": prompt}],
        tools=[generate_wikipedia_reading_list_tool],
        max_tokens=2000,
    )

    if response.stop_reason == "tool_use":
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input

        if tool_name == "generate_wikipedia_reading_list":
            generate_wikipedia_reading_list(tool_input["topic"], tool_input["article_titles"])
            print(f"tool used")
    else:
        print(f"Claude did not use the tool. Response: {response.content[0].text}")

In [35]:
get_research_help("animal consciousness", num_articles=3)
!cat output/research_reading.md

tool used
## Pirates across the world 
* [Piracy](https://en.wikipedia.org/wiki/Piracy) 
* [Piracy](https://en.wikipedia.org/wiki/Piracy) 
* [History of the Pittsburgh Pirates](https://en.wikipedia.org/wiki/History_of_the_Pittsburgh_Pirates) 
* [Barbary pirates](https://en.wikipedia.org/wiki/Barbary_pirates) 
* [Pirate code](https://en.wikipedia.org/wiki/Pirate_code) 


## history of hawaii 
* [Hawaii](https://en.wikipedia.org/wiki/Hawaii) 
* [Ancient Hawaii](https://en.wikipedia.org/wiki/Ancient_Hawaii) 
* [Hawaiian Kingdom](https://en.wikipedia.org/wiki/Hawaiian_Kingdom) 
* [Hawaiian Islands](https://en.wikipedia.org/wiki/Hawaiian_Islands) 
* [Hawaiian sovereignty movement](https://en.wikipedia.org/wiki/Hawaiian_sovereignty_movement) 
* [Hawaiian Renaissance](https://en.wikipedia.org/wiki/Hawaiian_Renaissance) 


## animal consciousness 
* [Animal consciousness](https://en.wikipedia.org/wiki/Animal_consciousness) 
* [Philosophy of mind](https://en.wikipedia.org/wiki/Philosophy_of_min